In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *

In [8]:
path = Path('imgs')

In [ ]:
path.ls()

In [ ]:
train = path/"train"

In [ ]:
train.ls()

In [ ]:
fns = get_image_files(train)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink)

In [ ]:
fns[0]

In [ ]:
def get_dls(bs, size):
    dBlock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items = get_image_files,
                       splitter = RandomSplitter(valid_pct = .2, seed=42),
                       get_y = parent_label,
                       item_tfms = Resize(640),
                       batch_tfms=aug_transforms(size=size, min_scale=0.75))
    return dBlock.dataloaders(train, num_workers=0, bs=bs)

In [ ]:
dls = get_dls(128, 128)
dls.train.show_batch(max_n=5, nrows=1)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=accuracy)

In [ ]:
learn.fine_tune(5, 1e-3)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
learn.validate()

In [ ]:
learn.export()

In [3]:
learn = load_learner("export.pkl")

In [9]:
test = path/"test"

In [10]:
test.ls()

(#79726) [Path('imgs/test/img_1.jpg'),Path('imgs/test/img_10.jpg'),Path('imgs/test/img_100.jpg'),Path('imgs/test/img_1000.jpg'),Path('imgs/test/img_100000.jpg'),Path('imgs/test/img_100001.jpg'),Path('imgs/test/img_100002.jpg'),Path('imgs/test/img_100003.jpg'),Path('imgs/test/img_100004.jpg'),Path('imgs/test/img_100005.jpg')...]

In [11]:
test_gif = get_image_files(test)
test_gif

(#79726) [Path('imgs/test/img_1.jpg'),Path('imgs/test/img_10.jpg'),Path('imgs/test/img_100.jpg'),Path('imgs/test/img_1000.jpg'),Path('imgs/test/img_100000.jpg'),Path('imgs/test/img_100001.jpg'),Path('imgs/test/img_100002.jpg'),Path('imgs/test/img_100003.jpg'),Path('imgs/test/img_100004.jpg'),Path('imgs/test/img_100005.jpg')...]

In [ ]:
failed_test_gif = verify_images(test_gif)
failed_test_gif

In [ ]:
failed_test_gif.map(Path.unlink)

In [12]:
batchSize = 64*2**1
batchSize

128

In [ ]:
dl = learn.dls.test_dl(test_gif, bs=batchSize)
predictions,  _ = learn.get_preds(dl=dl)

In [ ]:
import math

In [ ]:
from pathlib import Path
dfv = pd.read_csv("sample_submission.csv", index_col="img")

def EvaluateNumber(x):
    return math.floor(x * 10000) / 10000

i = 0
for test_img in test_gif:
    fileName = test_img.stem + test_img.suffix
    dfv.loc[fileName] = [EvaluateNumber(predictions[i][0].item()),
                         EvaluateNumber(predictions[i][1].item()),
                         EvaluateNumber(predictions[i][2].item()),
                         EvaluateNumber(predictions[i][3].item()),
                         EvaluateNumber(predictions[i][4].item()),
                         EvaluateNumber(predictions[i][5].item()),
                         EvaluateNumber(predictions[i][6].item()),
                         EvaluateNumber(predictions[i][7].item()),
                         EvaluateNumber(predictions[i][8].item()),
                         EvaluateNumber(predictions[i][9].item())]
    i = i + 1
    
dfv

In [ ]:
dfv.to_csv("submission.csv")